# Notebook used to clean the GageLoc.shp file
Source: https://www.sciencebase.gov/catalog/item/577445bee4b07657d1a991b6

This file has attributes that allow us to compare USGS gages to NHDPlus COMIDs

In [50]:
import geopandas as gpd
import numpy as np
import json
from shapely.vectorized import contains
import requests

%matplotlib inline

In [2]:
file = r'data/GageLoc/GageLoc.shp'

In [38]:
gdf = gpd.read_file(file)

In [39]:
gdf.head()

,COMID,EVENTDATE,REACHCODE,REACHSMDAT,REACHRESOL,FEATURECOM,FEATURECLA,SOURCE_ORI,SOURCE_DAT,SOURCE_FEA,FEATUREDET,Measure,Offset,EventType,FLComID,geometry
0,0,2014-12-04,01100005001603,None,Medium,0,0,"USGS, Water Resources Division",None,01206900,http://waterdata.usgs.gov/nwis/nwisman/?site_n...,45.31563,0.0,StreamGage,7712706,POINT Z (-73.06957627616134 41.67398586084777 0)
1,0,2014-12-04,01100005001605,None,Medium,0,0,"USGS, Water Resources Division",None,01206000,http://waterdata.usgs.gov/nwis/nwisman/?site_n...,15.86635,0.0,StreamGage,7713630,POINT Z (-73.06380259320338 41.70447942382566 0)
2,0,2014-12-04,01090002000178,None,Medium,0,0,"USGS, Water Resources Division",None,01105917,http://waterdata.usgs.gov/nwis/nwisman/?site_n...,9.65026,0.0,StreamGage,5878903,POINT Z (-70.83861945994248 41.6629127556689 0)
3,0,2014-12-04,01100005000020,None,Medium,0,0,"USGS, Water Resources Division",None,01205500,http://waterdata.usgs.gov/nwis/nwisman/?site_n...,45.47307,0.0,StreamGage,7718288,POINT Z (-73.16756586892285 41.3834695280468 0)
4,0,2014-12-04,01090004000089,None,Medium,0,0,"USGS, Water Resources Division",None,01109200,http://waterdata.usgs.gov/nwis/nwisman/?site_n...,69.19802,0.0,StreamGage,6129557,POINT Z (-71.25475652348776 41.8795729280263 0)


## Using a census states shapefile, assign state attributes to each GageLoc point

In [58]:
# a shapefile of states, to identify which points are in which state. Spatial reduction method
stateshp = r'C:\Data\Boundaries\cb_2017_us_state_500k\cb_2017_us_state_500k.shp'
usgdf = gpd.read_file(stateshp)
# CONUS only
usgdf = usgdf.drop(usgdf[(usgdf['NAME'] == 'Alaska')
                         | (usgdf['NAME'] == 'Guam')
                         | (usgdf['NAME'] == 'Commonwealth of the Northern Mariana Islands')
                         | (usgdf['NAME'] == 'American Samoa')
                         | (usgdf['NAME'] == 'United States Virgin Islands')].index)
assert usgdf.crs == gdf.crs, 'Error: the CRSs do not match'

In [59]:
usgdf.head()

,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry
0,54,01779805,0400000US54,54,WV,West Virginia,00,62265662566,489840834,"POLYGON ((-82.6431981036679 38.1690897960737, ..."
1,17,01779784,0400000US17,17,IL,Illinois,00,143784114293,6211277447,"POLYGON ((-91.512974 40.181062, -91.511073 40...."
2,24,01714934,0400000US24,24,MD,Maryland,00,25150696145,6980371026,"(POLYGON ((-76.05015299999999 37.986905, -76.0..."
3,16,01779783,0400000US16,16,ID,Idaho,00,214048160737,2393355752,"POLYGON ((-117.242675 44.396548, -117.234835 4..."
4,50,01779802,0400000US50,50,VT,Vermont,00,23873457570,1031134839,"POLYGON ((-73.43773999999999 44.045006, -73.43..."


### Pull the x and y coords into arrays for `shapely.vectorized.contains`

In [60]:
x = np.array(gdf.geometry.x)
y = np.array(gdf.geometry.y)

### Attribute the points with info based on the state that they are in

In [61]:
for i, g in enumerate(usgdf.geometry.tolist()):
    bmask = contains(g, x, y)
    geoid = usgdf.iloc[i]['GEOID']
    stusps = usgdf.iloc[i]['STUSPS']
    gdf['bool'] = bmask
    gdf.loc[gdf['bool'] == True, 'GEOID'] = geoid
    gdf.loc[gdf['bool'] == True, 'STUSPS'] = stusps
    gdf = gdf.drop(columns=['bool'])

### Attribute the points with 0/1 based on whether there is a USGS rating curve available at that STAID

In [57]:
for i, gage in enumerate(gdf.SOURCE_FEA.tolist()):
    url = f'https://waterdata.usgs.gov/nwisweb/get_ratings?site_no={gage}&file_type=exsa'
    if 'INDEP' in requests.get(url).text:
        gdf.loc[i, 'HAS_USGS_RC'] = 1
    else:
        gdf.loc[i, 'HAS_USGS_RC'] = 0

In [62]:
gdf.to_file('data/GageLoc/GageLoc_wState.shp')

### Make a dictionary containing STAID: COMID and write it to a json

In [6]:
subset = gdf.drop(columns=['COMID','EVENTDATE', 'REACHCODE', 'REACHSMDAT',
                        'REACHRESOL', 'FEATURECOM', 'FEATURECLA',
                        'SOURCE_ORI', 'SOURCE_DAT', 'FEATUREDET',
                        'Measure', 'Offset', 'EventType', 'geometry'])
usgs_nhd = dict(zip(subset.SOURCE_FEA, subset.FLComID))

In [7]:
usgs_nhd['01205500']

7718288

In [8]:
with open('data/STATID_COMID_dict.json', 'w') as fp:
    json.dump(usgs_nhd, fp, indent=2)

### Used STAID as the key because there are dups for the COMID (a NHD line can have more that one USGS gage station on it)

In [17]:
duped = subset.duplicated(subset='FLComID')
duplicated = subset[duped]

In [18]:
duplicated.head()

,SOURCE_FEA,FLComID
85,01208013,7713680
91,01189200,6109449
96,01189180,6109449
97,01184455,7700004
110,01196210,6174472
